In [ ]:
import pandas as pd
import folium
import random

# Load the datasets
positions = pd.read_csv('../data/vessel_positions.csv')
events = pd.read_csv('../data/simulated_vessel_proximity_events.csv')

# Convert timestamps to readable format
positions['datetime'] = pd.to_datetime(positions['t'], unit='ms')
events['datetime'] = pd.to_datetime(events['t'], unit='ms')

# Helper: generate distinct colors for each vessel
def generate_color():
    r = lambda: random.randint(0, 255)
    return f'#{r():02X}{r():02X}{r():02X}'

# Initialize base map
m = folium.Map(location=[38.2, 24.5], zoom_start=7, tiles='CartoDB Positron')

# Plot vessel routes with colored lines and point dots
for vessel_id in positions['vessel_id'].unique():
    vessel_data = positions[positions['vessel_id'] == vessel_id].sort_values('datetime')
    coords = list(zip(vessel_data['lat'], vessel_data['lon']))
    color = generate_color()

    # Add PolyLine (route line)
    folium.PolyLine(coords, color=color, weight=2.5, opacity=0.8,
                    popup=f'Vessel ID: {vessel_id}').add_to(m)

    # Add PolyLine with zero weight and small dots
    for coord in coords:
        folium.PolyLine(
            locations=[coord, coord],  # Same start and end point to simulate a dot
            color=color,
            weight=1.5,  # Very thin to appear like a dot
            opacity=0.8
        ).add_to(m)

# Add lightweight dot markers with timestamp on hover
#    for _, row in vessel_data.iterrows():
#        folium.CircleMarker(
#            location=[row['lat'], row['lon']],
#            radius=2,  # Smaller dots for better visibility at zoomed-out levels
#            color=color,
#            fill=True,
#            fill_opacity=0.7,
#            popup=f"<b>Vessel:</b> {vessel_id}<br><b>Time:</b> {row['datetime']}",
#        ).add_to(m)

# Mark proximity events (collision warnings) as alert icons
for _, row in events.iterrows():
    folium.Marker(
        location=[row['lat'], row['lon']],
        icon=folium.Icon(color='red', icon='exclamation-triangle', prefix='fa'),
        popup=f"<b>🚨 Proximity Event</b><br>Vessels: {row['vessel_id1']} & {row['vessel_id2']}<br>{row['datetime']}"
    ).add_to(m)

# Save and display map
m.save('maritime_collision_visual_map.html')
m
